In [132]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pylab as plt
import pandas as pd

In [199]:
- np.log(1 - 0.99)

4.605170185988091

In [207]:
class I:
    U = 0
    FL = 1
    SL = 2
    I = 3
    Tx = 4
    
    
class A:
    Inc = 0
    Mor = 1
    Covered = 2
    Eli = 3
    Yield = 4
    PCF = 5

    
    
class ModelTB:
    def get_y0(self):
        y0 = np.array([[970, 10, 10, 10, 0], np.zeros(5)]).T / 1000
        a0 = np.zeros(6)
        return np.concatenate([y0.reshape(-1), a0])
    
    def intervene(self, t, y, dy, da, acf):
        if acf['Cohort']:
            r_acf = - np.log(1 - acf['Coverage'])
            r_acf = min(r_acf, 20)
        else:
            r_acf = acf['Coverage']
        switch = acf['Switch']
        acf = r_acf * y[:, 0]
        eli = acf * np.array([1, 1, 1, 1, 0])
        
        da[A.Covered] += acf.sum()
        da[A.Eli] += eli.sum()
        da[A.Yield] += eli[I.I]
        
        if switch:
            dy[:, 0] -= eli
            dy[0:3, 1] += eli[0:3]
            dy[4, 1] += eli[3]
            dy[:, 0] += y[:, 1]
            dy[:, 1] -= y[:, 1]
        else:
            dy[3, 0] -= eli[3]
            dy[4, 1] += eli[3]
        
        return dy, da
    
    def __call__(self, t, ya, pars, acf=None):
        y, aux = ya[:-6], ya[-6:]
        y = y.reshape((5, 2))
        
        foi = pars['beta'] * y[I.I].sum()
        lat = pars['r_lat'] * y[I.FL]
        act = pars['r_act'] * y[I.FL]
        react = pars['r_react'] * y[I.SL]

        det = pars['r_det'] * y[I.I]
        txo = pars['r_tx'] * y[I.Tx]

        die_tb = pars['r_die_tb'] * y[I.I]
        r_die_bg = pars['r_die_bg'] - die_tb.sum() / y.sum()
        r_die_bg = max(r_die_bg, 0)
        die_bg = r_die_bg * y
        sc = pars['r_sc'] * y[I.I]

        dy = np.zeros_like(y)
        
        dy[I.U] -= foi * y[I.U]
        dy[I.FL] += foi * (y[I.U] + (1 - pars['p_im']) * y[I.SL]) - act - lat
        dy[I.SL] += lat - foi * (1 - pars['p_im']) * y[I.SL] + txo + sc - react
        dy[I.I] += act + react - det - sc
        dy[I.Tx] += det - txo

        dy -= die_bg
        dy[I.I] -= die_tb
        dy[I.U, 0] += die_bg.sum() + die_tb.sum()

        da = np.zeros_like(aux)
        da[A.Inc] += (act + react).sum()
        da[A.Mor] += die_tb.sum()
        
        if acf is not None:
            dy, da = self.intervene(t, y, dy, da, acf)
        
        return np.concatenate([dy.reshape(-1), da.reshape(-1)])
    
    def mea(self, t, ya):
        aux = ya[-6:]

        return {
            'Time': t,
            'Inc': aux[A.Inc],
            'Mor': aux[A.Mor],
            'Covered': aux[A.Covered],
            'Eli': aux[A.Eli],
            'Yield': aux[A.Yield],
            'PCF': aux[A.PCF]
        }

In [208]:
p = {
    'beta': 15,
    'r_act': 0.5 * 0.1 / 0.9,
    'r_lat': 0.5,
    'r_react': 0.002,
    'r_det': 1,
    'r_tx': 2,
    'r_die_tb': 0.1,
    'r_sc': 0.2,
    'r_die_bg': 0.05,
    'p_im': 0.6
}

In [209]:
model = ModelTB()
y0 = model.get_y0()

sol = solve_ivp(model, t_span = [0, 500], y0 = y0, args=(p, ))
ya0 = sol.y[:, -1]
ya0[-6:] = 0

In [210]:

def exp_discrete(model, ya0, coverage=0.1, interval=0.5):
    ts = np.linspace(0, 10, int(10 / interval) + 1)
    ya = ya0
    mss = list()
    mss.append(model.mea(0, ya))

    for t0, t1 in zip(ts[:-1], ts[1:]):
        sol = solve_ivp(model, t_span = [t0, t1], y0 = ya, args=(p, ))
        ya = sol.y[:, -1]

        ## ACF
        cov = coverage * interval
        y, aux = ya[:-6], ya[-6:]
        y = y.reshape((-1, 2))
        prop =  cov / y[:, 0].sum()
        prop = min(prop, 1)
        screened = y[:, 0] * prop
        eli = screened * np.array([1, 1, 1, 1, 0])
        found = eli[I.I]
        y[:, 0] -= eli
        y[0:3, 1] += eli[0:3]
        y[4, 1] += eli[3]
        aux[A.Screened] += screened.sum()
        aux[A.Eli] += eli.sum()
        aux[A.Yield] += eli[3]
        ya = np.concatenate([y.reshape(-1), aux])

        if round(t1) == t1:
            y, aux = ya[:-6], ya[-6:]
            y = y.reshape((-1, 2))
            y[:, 0] += y[:, 1]
            y[:, 1] = 0
            ya = np.concatenate([y.reshape(-1), aux])
            mss.append(model.mea(t1, ya))

    mss = pd.DataFrame(mss).assign(Coverage = coverage, Interval = interval)
    return mss

In [211]:
acf = {'Coverage': 0.1, 'Switch': True, 'Cohort': False}

ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, acf), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)]).assign(**acf)

,Time,Inc,Mor,Covered,Eli,Yield,PCF,Coverage,Switch,Cohort
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.1,True,False
1,1.0,0.002919,0.000210,0.096475,0.096370,0.000208,0.0,0.1,True,False
2,2.0,0.005789,0.000412,0.189422,0.189216,0.000403,0.0,0.1,True,False
3,3.0,0.008607,0.000610,0.281253,0.280949,0.000591,0.0,0.1,True,False
4,4.0,0.011378,0.000804,0.372729,0.372330,0.000775,0.0,0.1,True,False
5,5.0,0.014108,0.000995,0.464092,0.463599,0.000955,0.0,0.1,True,False
6,6.0,0.016803,0.001183,0.555421,0.554836,0.001133,0.0,0.1,True,False
7,7.0,0.019467,0.001369,0.646738,0.646062,0.001309,0.0,0.1,True,False
8,8.0,0.022105,0.001553,0.738051,0.737286,0.001483,0.0,0.1,True,False
9,9.0,0.024718,0.001735,0.829363,0.828510,0.001655,0.0,0.1,True,False


In [213]:
acf = {'Coverage': 0.1, 'Switch': False, 'Cohort': True}

ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, acf), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)]).assign(**acf)

,Time,Inc,Mor,Covered,Eli,Yield,PCF,Coverage,Switch,Cohort
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.1,False,True
1,1.0,0.002919,0.000210,0.105349,0.105239,0.000221,0.0,0.1,False,True
2,2.0,0.005784,0.000411,0.210676,0.210461,0.000433,0.0,0.1,False,True
3,3.0,0.008593,0.000607,0.315984,0.315667,0.000639,0.0,0.1,False,True
4,4.0,0.011350,0.000799,0.421273,0.420856,0.000841,0.0,0.1,False,True
5,5.0,0.014063,0.000987,0.526545,0.526031,0.001040,0.0,0.1,False,True
6,6.0,0.016736,0.001173,0.631801,0.631190,0.001235,0.0,0.1,False,True
7,7.0,0.019375,0.001356,0.737042,0.736337,0.001428,0.0,0.1,False,True
8,8.0,0.021985,0.001537,0.842269,0.841471,0.001618,0.0,0.1,False,True
9,9.0,0.024568,0.001715,0.947484,0.946593,0.001806,0.0,0.1,False,True


In [194]:
ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, {'Coverage': 0.1, 'Switch': False}), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)])

,Time,Inc,Mor,Screened,Eli,Yield,PCF
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1.0,0.002920,0.000210,0.099990,0.099885,0.000210,0.0
2,2.0,0.005788,0.000412,0.199960,0.199755,0.000412,0.0
3,3.0,0.008603,0.000609,0.299912,0.299610,0.000609,0.0
4,4.0,0.011369,0.000802,0.399847,0.399450,0.000802,0.0
5,5.0,0.014091,0.000992,0.499767,0.499277,0.000992,0.0
6,6.0,0.016777,0.001179,0.599672,0.599090,0.001179,0.0
7,7.0,0.019430,0.001364,0.699564,0.698892,0.001363,0.0
8,8.0,0.022054,0.001546,0.799444,0.798681,0.001545,0.0
9,9.0,0.024653,0.001726,0.899311,0.898460,0.001725,0.0


In [176]:
res = list()

for c in [0.1, 0.25, 0.5, 0.75, 1]:
    for i in [1, 0.5, 0.25, 0.1, 0.05, 0.1]:
        res.append(exp_discrete(model, ya0, coverage=c, interval=i))

res = pd.concat(res)
res

,Time,Inc,Mor,Screened,Eli,Yield,PCF,Coverage,Interval
0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.1,1.0
1,1.0,0.002934,0.000217,0.1,0.099894,0.000217,0.0,0.1,1.0
2,2.0,0.005829,0.000422,0.2,0.199789,0.000427,0.0,0.1,1.0
3,3.0,0.008667,0.000620,0.3,0.299688,0.000631,0.0,0.1,1.0
4,4.0,0.011451,0.000814,0.4,0.399588,0.000831,0.0,0.1,1.0
...,...,...,...,...,...,...,...,...,...
6,6.0,0.011791,0.000490,6.0,5.991950,0.006916,0.0,1.0,0.1
7,7.0,0.013062,0.000538,7.0,6.991153,0.007583,0.0,1.0,0.1
8,8.0,0.014247,0.000581,8.0,7.990421,0.008201,0.0,1.0,0.1
9,9.0,0.015361,0.000623,9.0,8.989740,0.008782,0.0,1.0,0.1


In [177]:
res.to_csv('out/test.csv')